In [1]:
import os
!pip install llama-index python-dotenv

In [2]:
from llama_index.core.query_pipeline import QueryPipeline

chinook database をサンプルデータとして利用する。

In [3]:
%pip install llama-index-llms-openai

Note: you may need to restart the kernel to use updated packages.


In [4]:
!curl "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O ./chinook.zip
!unzip -o ./chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0   234k      0  0:00:01  0:00:01 --:--:--  234k
curl: (6) Could not resolve host: .
Archive:  ./chinook.zip
  inflating: chinook.db              


In [5]:
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")
sql_database = SQLDatabase(engine)

global callback manager をセットアップする。

In [6]:
from llama_index.core.settings import Settings
from llama_index.core.callbacks import CallbackManager

callback_manager = CallbackManager()
Settings.callback_manager = callback_manager

In [7]:
from dotenv import load_dotenv

load_dotenv()

True

setup a simple text-to-SQL tool: given a query, translate text to SQL, execute against database, and get back a result.

In [8]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True
)

sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    )
)

次に、クエリ パイプライン構文を使用して、単一ステップの ReAct パイプラインをセットアップします。 これは、次のことを行う複数の部分からなるプロセスです。

1. エージェントの入力を取り込む
2. LLM を使用して ReAct プロンプトを呼び出し、次のアクション/ツールを生成します (または応答を返します)。
3. ツール/アクションが選択されている場合は、ツール パイプラインを呼び出してツールを実行し、応答を収集します。
4. レスポンスが発生した場合はレスポンスを取得します。

この全体を通じて、エージェント固有のさまざまなクエリ コンポーネントを使用します。 通常のクエリ パイプラインとは異なり、これらは QueryPipelineAgentWorker で使用されるクエリ パイプライン用に特別に設計されています。

- AgentInputComponent は、エージェント入力 (タスク、状態ディクショナリ) をクエリ パイプラインの入力セットに変換できるようにします。
- AgentFnComponent: 現在のタスク、状態、および任意の入力を取り込み、出力を返すことができる汎用プロセッサです。 このクックブックでは、ReAct プロンプトをフォーマットする関数コンポーネントを定義します。 ただし、これはどこにでも置くことができます。
- [このノートブックでは使用されません] CustomAgentComponent: AgentFnComponent と同様に、_run_component を実装して、タスクと状態にアクセスできる独自のロジックを定義できます。 AgentFnComponent よりも冗長ですが、より柔軟です (たとえば、初期化変数を定義でき、コールバックは基本クラス内にあります)。
AgentFnComponent および AgentInputComponent に渡される関数には、タスクと状態がエージェントから渡される入力であるため、入力変数としてタスクと状態を含める必要があることに注意してください。

エージェント クエリ パイプラインの出力は Tuple[AgentChatResponse, bool] でなければならないことに注意してください。 これは以下でわかります。

In [9]:
from llama_index.core.query_pipeline import QueryPipeline as QP

qp = QP(verbose=True)

ここでは、すべてのエージェント ステップの開始時に呼び出されるエージェント入力コンポーネントを定義します。 入力を渡すだけでなく、初期化/状態変更も行います。

In [10]:
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep
)
from llama_index.core.agent import Task, AgentChatResponse
from llama_index.core.query_pipeline import (
    AgentInputComponent,
    AgentFnComponent,
    CustomAgentComponent,
    QueryComponent,
    ToolRunnerComponent
)
from llama_index.core.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast


## Agent Input Component
## This is the component that produces agent inputs to the rest of the components can also put initialization logic here.
def agent_input_fn(task: Task, state: Dict[str, Any]) -> Dict[str, Any]:
    """Agent input function.
    
    Returns:
        A Dictionary of output keys and values. If you are specifying src_key when defining links between this component and other components, make sure the src_key matches the specified output_key.

    """
    # initialize current_reasoning
    if "current_reasoning" not in state:
        state["current_reasoning"] = []
    reasoning_step = ObservationReasoningStep(observation=task.input)
    state["current_reasoning"].append(reasoning_step)
    return {"input": task.input}


agent_input_component = AgentInputComponent(fn=agent_input_fn)

ここでは、ReAct プロンプトを生成するエージェント コンポーネントを定義し、LLM から出力が生成された後、構造化オブジェクトに解析します。

In [11]:
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.query_pipeline import InputComponent, Link
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool


## define prompt function
def react_prompt_fn(
    task: Task, state: Dict[str, Any], input: str, tools: List[BaseTool]
) -> List[ChatMessage]:
    # Add input to reasoning
    chat_formatter = ReActChatFormatter()
    return chat_formatter.format(
        tools,
        chat_history=task.memory.get() + state["memory"].get_all(),
        current_reasoning=state["current_reasoning"]
    )

react_prompt_component = AgentFnComponent(
    fn=react_prompt_fn, partial_dict={"tools": [sql_tool]}
)

In [12]:
from llama_index.core.base.llms.generic_utils import messages_to_prompt

chat_formatter = ReActChatFormatter()
msgs = chat_formatter.format(
    [sql_tool],
    chat_history=[],
    current_reasoning=[]
)
print(messages_to_prompt(msgs))

system: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
> Tool Name: sql_tool
Tool Description: Useful for translating a natural language query into a SQL query
Tool Args: {"type": "object", "properties": {"input": {"title": "Input", "type": "string"}}, "required": ["input"]}



## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of sql_tool) if using a tool.
Action Input: the input to the tool, in a JSON format represent

LLM が出力を与えると、決定木が得られます。

1. 答えが得られれば、それで終わりです。 出力を処理する
2. アクションが指定されている場合は、指定された引数を使用して指定されたツールを実行し、出力を処理する必要があります。

ツールの呼び出しは、ToolRunnerComponent モジュールを介して実行できます。 これはツールのリストを取り込む単純なラッパー モジュールで、指定されたツール名 (すべてのツールには名前があります) とツール アクションで「実行」できます。

CustomAgentComponent をサブクラス化するこの全体的なモジュール OutputAgentComponent を実装します。

注: また、高レベルのコールバック マネージャーを介してツール ランナー サブモジュールに渡すために、sub_query_components も実装します。

In [13]:
from typing import Set, Optional
from llama_index.core.agent.react.output_parser import ReActOutputParser
from llama_index.core.llms import ChatResponse
from llama_index.core.agent.types import Task

def parse_react_output_fn(
        task: Task, state: Dict[str, Any], chat_response: ChatResponse
):
    """Parse ReAct output into a reasoning step."""
    output_parser = ReActOutputParser()
    reasoning_step = output_parser.parse(chat_response.message.content)
    return {"done": reasoning_step.is_done, "reasoning_step": reasoning_step}

parse_react_output = AgentFnComponent(fn=parse_react_output_fn)

def run_tool_fn(
        task: Task, state: Dict[str, Any], reasoning_step: ActionReasoningStep
):
    """Run tool and process tool output."""
    tool_runner_component = ToolRunnerComponent(
        [sql_tool],
        callback_manager=task.callback_manager
    )
    tool_output = tool_runner_component.run_component(
        tool_name=reasoning_step.action,
        tool_input=reasoning_step.action_input
    )
    observation_step = ObservationReasoningStep(observation=str(tool_output))
    state["current_reasoning"].append(observation_step)
    
    return {"response_str": observation_step.get_content(), "is_done": False}
    
run_tool = AgentFnComponent(fn=run_tool_fn)

def process_response_fn(
        task: Task, state: Dict[str, Any], response_step: ResponseReasoningStep
):
    """Process response."""
    state["current_reasoning"].append(response_step)
    response_str = response_step.response
    # Now that we are done with this step, put into memory
    state["memory"].put(
        ChatMessage(
            content=task.input,
            role=MessageRole.USER
    ))
    state["memory"].put(
        ChatMessage(
            content=response_str,
            role=MessageRole.ASSISTANT
        )
    )
    
    return {"response_str": response_str, "is_done": True}

process_response = AgentFnComponent(fn=process_response_fn)

def process_agent_response_fn(
        task: Task, state: Dict[str, Any], response_dict: dict
):
    """Process agent response."""
    return (
        AgentChatResponse(response_dict["response_str"]),
        response_dict["is_done"]
    )
    
process_agent_response = AgentFnComponent(fn=process_agent_response_fn)

これで、最上位のエージェント パイプライン (agent_input -> accept_prompt -> llm -> accept_output) をつなぎ合わせることができます。

最後のコンポーネントは、サブコンポーネントを呼び出す if-else コンポーネントです。

In [14]:
from llama_index.llms.openai import OpenAI

qp.add_modules(
    {
        "agent_input": agent_input_component,
        "react_prompt": react_prompt_component,
        "llm": OpenAI(model=os.getenv("CHAT_MODEL_GPT4")),
        "react_output_parser": parse_react_output,
        "run_tool": run_tool,
        "process_response": process_response,
        "process_agent_response": process_agent_response,
    }
)

In [15]:
# Link input to react prompt to parsed out response (either tool action/input or observation)
qp.add_chain(
    ["agent_input", "react_prompt", "llm", "react_output_parser"]
)

# Add conditional link from react output to tool call (if not done)
# ReActの結果、Tool呼び出しがある場合
qp.add_link(
    "react_output_parser",
    "run_tool",
    condition_fn=lambda x: not x["done"],
    input_fn=lambda x: x["reasoning_step"]
)

# Add conditional link from react output to final response processing (if done)
# ReActの結果、終了の場合
qp.add_link(
    "react_output_parser",
    "process_response",
    condition_fn=lambda x: x["done"],
    input_fn=lambda x: x["reasoning_step"]
)

# Whether response processing or tool output processing, add link to final agent response
qp.add_link("process_response", "process_agent_response")
qp.add_link("run_tool", "process_agent_response")

In [16]:
from pyvis.network import Network

net = Network(
    notebook=True,
    cdn_resources="in_line",
    directed=True
)
net.from_nx(qp.clean_dag)
net.show("agent_dag.html")

agent_dag.html


これは、テキストから SQL へのクエリ パイプラインを中心にエージェントをセットアップする方法です。

In [17]:
from llama_index.core.agent import QueryPipelineAgentWorker, AgentRunner
from llama_index.core.callbacks import CallbackManager

agent_worker = QueryPipelineAgentWorker(qp)
agent = AgentRunner(
    agent_worker=agent_worker,
    callback_manager=CallbackManager([]),
    verbose=True
)

Agentを動かす

In [18]:
# Start task
task = agent.create_task(
    "What are some tracks from the artist AC/DC? Limit it to 3."
)

In [19]:
step_output = agent.run_step(task.task_id)

> Running step 328a8222-b2a9-46f0-b360-89acb067f87f. Step input: What are some tracks from the artist AC/DC? Limit it to 3.
> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='f67dd681-1f91-40f7-acf2-0805b12109e5' input='What are some tracks from the artist AC/DC? Limit it to 3.' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3.

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\n...

> Running module react_output_parser with input: 
cha

In [20]:
step_output.is_last

False

In [27]:
print(step_output)

Three tracks from the artist AC/DC are "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".


In [21]:
step_output = agent.run_step(task.task_id)

> Running step 2be3f540-66b2-4273-97de-bb95bbb0e93e. Step input: None
> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='f67dd681-1f91-40f7-acf2-0805b12109e5' input='What are some tracks from the artist AC/DC? Limit it to 3.' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3.

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\n...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: The user has repeated 

In [22]:
step_output.is_last

True

In [23]:
response = agent.finalize_response(task.task_id)

In [24]:
print(str(response))

Three tracks from the artist AC/DC are "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".


In [25]:
# Run this e2e
agent.reset()
response = agent.chat(
    "What are some tracks from the artist AC/DC? Limit it to 3."
)

> Running step 6e530307-88d7-407d-9234-5886dca847cf. Step input: What are some tracks from the artist AC/DC? Limit it to 3.
> Running module agent_input with input: 
state: {'sources': [], 'memory': ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method Encoding.encode of <Encoding 'cl100k_base'>>, allowed_special='all'), chat_store=SimpleChatSto...
task: task_id='614e2fa9-823b-4891-b5a7-fdfe38134a95' input='What are some tracks from the artist AC/DC? Limit it to 3.' memory=ChatMemoryBuffer(token_limit=3000, tokenizer_fn=functools.partial(<bound method...

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3.

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, content='You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.\n\n## Tools\n\n...

> Running module react_output_parser with input: 
cha

In [26]:
print(step_output)

Three tracks from the artist AC/DC are "For Those About To Rock (We Salute You)", "Put The Finger On You", and "Let's Get It Up".
